Import BeautifulSoup and requests to extract html document from a url 

## All required imports

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import io #geocoder # import geocoder

import numpy as np # library to handle data in a vectorized manner

#import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

#import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
#!conda install -c conda-forge scikit-learn 
#from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: | 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/noarch::ibm-wsrt-py37main-main==custom=2155
  - conda-forge/linux-64::pytorch==1.8.0=cpu_py37ha70c682_1
  - defaults/noarch::ibm-wsrt-py37main-keep==0.0.0=2155
done

# All requested packages already installed.

Solving environment: \ 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/noarch::ibm-wsrt-py37main-main==custom=2155
  - conda-forge/linux-64::pytorch==1.8.0=cpu_py37ha70c682_1
  - defaults/noarch::ibm-wsrt-py37main-keep==0.0.0=2155
done

# All requested packages already installed.

Libraries imported.


## Get links

In [2]:
url="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv"
r=requests.get(url).content
pc=pd.read_csv(io.StringIO(r.decode('utf-8')))
pc.head(5)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Get BeautifulSoup object from url

In [3]:
req = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(req.content,'lxml')

Extract table containing postal code, borough and neighborhood

In [4]:
table=soup.find('table')

Generate a list to store information in. From the entire table, extract postal code, borough and neighborhood. Only extract information when boroughs are not 'Not assigned'. 
Since the table from the BeautifulSoup object is listed in rows we can loop through the rows to extract the information one by one and append the table_contents list by the cell objects.

In [5]:
table_contents=[]

for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)



Turn the list into a dataframe

In [6]:
# print(table_contents)
df=pd.DataFrame(table_contents)


Replace faulty borough information 

In [7]:
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

Display the shape of the dataframe

In [8]:
df.shape

(103, 3)

In [9]:
df.head(5)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [10]:
df = df.merge(pc, how='left', left_on='PostalCode', right_on='Postal Code')

In [11]:
del df['Postal Code']

In [12]:
df.head(20)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


Get geographical data for first mapping of neighborhoods

In [16]:
address = 'Toronto'

geolocator = Nominatim(user_agent="tr_exp")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


## Mapping of Toronto Neighborhoods

In [17]:
#map of toronto
map_tr = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_tr)  
    
map_tr

### Only include boroughs that have york in name

In [18]:
tr_york = df[df['Borough'].str.contains("York")].reset_index(drop=True)
tr_york.head(30)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
3,M3B,North York,Don Mills North,43.745906,-79.352188
4,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
5,M6B,North York,Glencairn,43.709577,-79.445073
6,M3C,North York,Don Mills South,43.725900,-79.340923
7,M4C,East York,Woodbine Heights,43.695344,-79.318389
8,M6C,York,Humewood-Cedarvale,43.693781,-79.428191
9,M6E,York,Caledonia-Fairbanks,43.689026,-79.453512


## Map again, this time only boroughs with york in name

In [19]:
# create map of the yorks using latitude and longitude values
map_yorks = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(tr_york['Latitude'], tr_york['Longitude'], tr_york['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_yorks)  
    
map_yorks

## Counting of unique neighborhoods in boroughs with York in the name

As can be seen, the borough of East York / East Toronto only has one neighborhood, while the borough North York has 24

In [20]:
df_test = tr_york.groupby('Borough')['Neighborhood'].nunique()

print (df_test)

Borough
East York                  4
East York/East Toronto     1
North York                24
York                       5
Name: Neighborhood, dtype: int64


### Boroughs are numbered for mapping preparation and then mapped based on their numbering

In [21]:
tr_york_br = tr_york
# create a list of our conditions
conditions = [
    (tr_york_br['Borough'] == "York"),
    (tr_york_br['Borough'] == "East York"),
    (tr_york_br['Borough'] == "North York"),
    (tr_york_br['Borough'] == "East York/East Toronto")
    ]

# create a list of the values we want to assign for each condition
values = [4, 3, 2, 1]

# create a new column and use np.select to assign values to it using our lists as arguments
tr_york_br['br_n'] = np.select(conditions, values)

# display updated DataFrame
tr_york_br.head()


,PostalCode,Borough,Neighborhood,Latitude,Longitude,br_n
0,M3A,North York,Parkwoods,43.753259,-79.329656,2
1,M4A,North York,Victoria Village,43.725882,-79.315572,2
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,2
3,M3B,North York,Don Mills North,43.745906,-79.352188,2
4,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,3


In [22]:
n_br = 4

# create map
map_bor = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the boroughs
x = np.arange(n_br)
ys = [i + x + (i*x)**2 for i in range(n_br)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, br in zip(tr_york_br['Latitude'], tr_york_br['Longitude'], tr_york_br['Neighborhood'], tr_york_br['br_n']):
    label = folium.Popup(str(poi) + ' Borough ' + str(br), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[br-1],
        fill=True,
        fill_color=rainbow[br-1],
        fill_opacity=0.7).add_to(map_bor)
       
map_bor

As can be seen, East York / East Toronto lies in the South East close to the river (purple dot)

In [23]:
CLIENT_ID = 'QW0D2R4LUIW15O4XC0SSZ1DOCBDKSQGU0ZM24Y25JUGYOPDB' # your Foursquare ID
CLIENT_SECRET = '3XE0G1EBHQQ0CZ40CPWKWW2ZIVTA2ACXAUDATE05UN2DZUWE' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: QW0D2R4LUIW15O4XC0SSZ1DOCBDKSQGU0ZM24Y25JUGYOPDB
CLIENT_SECRET:3XE0G1EBHQQ0CZ40CPWKWW2ZIVTA2ACXAUDATE05UN2DZUWE


### Neighborhood data is determined

In [24]:
neighborhood_latitude = tr_york_br.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = tr_york_br.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = tr_york_br.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))


Latitude and longitude values of Parkwoods are 43.7532586, -79.3296565.


### 2000 metres radius around the neighborhood location is chosen, url for foursquare request is generated

In [25]:
# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 2000 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=QW0D2R4LUIW15O4XC0SSZ1DOCBDKSQGU0ZM24Y25JUGYOPDB&client_secret=3XE0G1EBHQQ0CZ40CPWKWW2ZIVTA2ACXAUDATE05UN2DZUWE&v=20180605&ll=43.7532586,-79.3296565&radius=2000&limit=100'

In [26]:
results_york = requests.get(url).json()
results_york

{'meta': {'code': 200, 'requestId': '609068090e6a173654a77b06'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Parkwoods - Donalda',
  'headerFullLocation': 'Parkwoods - Donalda, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 89,
  'suggestedBounds': {'ne': {'lat': 43.77125861800002,
    'lng': -79.30478345939711},
   'sw': {'lat': 43.735258581999986, 'lng': -79.35452954060288}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b8991cbf964a520814232e3',
       'name': "Allwyn's Bakery",
       'location': {'address': '81 Underhill drive',
        'lat': 43.75984035203157,
        'lng': -79.32471879917513,
        'labeledLatLngs': [{'label': 'display'

## Now that the locations of interest are found, the data is prepared so that the venues can be analyzed

In [27]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [28]:
venues = results_york['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
nearby_venues.head()


/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,Allwyn's Bakery,"[{'id': '4bf58dd8d48988d144941735', 'name': 'C...",43.759840,-79.324719
1,Tim Hortons,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",43.760668,-79.326368
2,Donalda Golf & Country Club,"[{'id': '4bf58dd8d48988d1e6941735', 'name': 'G...",43.752816,-79.342741
3,Graydon Hall Manor,"[{'id': '4bf58dd8d48988d171941735', 'name': 'E...",43.763923,-79.342961
4,Island Foods,"[{'id': '4bf58dd8d48988d144941735', 'name': 'C...",43.745866,-79.346035


In [29]:
test_lst = nearby_venues['venue.categories']
testset = test_lst[0]
testset
pd_test = pd.DataFrame(testset)
pd_test['name']
pd_test

,id,name,pluralName,shortName,icon,primary
0,4bf58dd8d48988d144941735,Caribbean Restaurant,Caribbean Restaurants,Caribbean,{'prefix': 'https://ss3.4sqi.net/img/categorie...,True


## The name of the venue category is extracted through the function here under

In [30]:
def get_name(dframe):
    for index, row in dframe.iterrows(): 
        lst = dframe['venue.categories']
        name_df = pd.DataFrame(lst[index])
        name = name_df['name'][0]
        dframe['venue.categories'][index] = name
    return dframe
    

In [31]:
nearby_venues['venue.categories']
lst_names = get_name(nearby_venues)
nearby_venues_tr = lst_names
nearby_venues_tr.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,Allwyn's Bakery,Caribbean Restaurant,43.759840,-79.324719
1,Tim Hortons,Café,43.760668,-79.326368
2,Donalda Golf & Country Club,Golf Course,43.752816,-79.342741
3,Graydon Hall Manor,Event Space,43.763923,-79.342961
4,Island Foods,Caribbean Restaurant,43.745866,-79.346035


The venue categories are found and can be counted

In [33]:
print('{} venues were returned by Foursquare.'.format(nearby_venues_tr.shape[0]))

89 venues were returned by Foursquare.


### As in the New York example, nearby venues are determined

In [34]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results_york = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results_york])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [35]:
tr_york_venues = getNearbyVenues(names=tr_york_br['Neighborhood'],
                                   latitudes=tr_york_br['Latitude'],
                                   longitudes=tr_york_br['Longitude']
                                  )

Parkwoods
Victoria Village
Lawrence Manor, Lawrence Heights
Don Mills North
Parkview Hill, Woodbine Gardens
Glencairn
Don Mills South
Woodbine Heights
Humewood-Cedarvale
Caledonia-Fairbanks
Leaside
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Fairview, Henry Farm, Oriole
Northwood Park, York University
The Danforth  East
Bayview Village
Downsview East
York Mills, Silver Hills
Downsview West
North Park, Maple Leaf Park, Upwood Park
Humber Summit
Willowdale, Newtonbrook
Downsview Central
Bedford Park, Lawrence Manor East
Del Ray, Mount Dennis, Keelsdale and Silverthorn
Humberlea, Emery
Willowdale South
Downsview Northwest
Runnymede, The Junction North
Weston
York Mills West
Willowdale West


In [36]:
print(tr_york_venues.shape)
tr_york_venues.head()

(337, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


In [37]:
tr_york_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Bathurst Manor, Wilson Heights, Downsview North",21,21,21,21,21,21
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",26,26,26,26,26,26
Caledonia-Fairbanks,4,4,4,4,4,4
"Del Ray, Mount Dennis, Keelsdale and Silverthorn",4,4,4,4,4,4
Don Mills North,5,5,5,5,5,5
Don Mills South,19,19,19,19,19,19
Downsview Central,4,4,4,4,4,4
Downsview East,3,3,3,3,3,3


## The counted venues by neighborhood can be extracted from the table above. As can be seen, Fairview, Henry Farm and Oriole have the most venues (68), followed by Willowdale South (34) and Leaside (31). Only 3 neighborhoods have only 1 venue.

In [38]:
print('There are {} uniques categories.'.format(len(tr_york_venues['Venue Category'].unique())))

There are 118 uniques categories.


After it is found that there are 118 categories, they are attributed to the neighborhoods as shown below

In [39]:
# one hot encoding
tr_york_onehot = pd.get_dummies(tr_york_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
tr_york_onehot['Neighborhood'] = tr_york_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [tr_york_onehot.columns[-1]] + list(tr_york_onehot.columns[:-1])
tr_york_onehot = tr_york_onehot[fixed_columns]

tr_york_onehot.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Store,Bike Shop,Boutique,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Business Service,Butcher,Café,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Curling Ice,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Electronics Store,Falafel Restaurant,Fast Food Restaurant,Field,Fish & Chips Shop,Flea Market,Food & Drink Shop,Food Court,Food Service,Food Truck,Fried Chicken Joint,Furniture / Home Store,Gas Station,Gastropub,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health Food Store,Hobby Shop,Hockey Arena,Home Service,Hotel,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Korean Restaurant,Liquor Store,Lounge,Luggage Store,Massage Studio,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Movie Theater,Moving Target,Park,Pet Store,Pharmacy,Pizza Place,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Sandwich Place,Shoe Store,Shopping Mall,Skating Rink,Smoothie Shop,Spa,Sporting Goods Shop,Sports Bar,Supermarket,Sushi Restaurant,Thai Restaurant,Theater,Toy / Game Store,Trail,Video Game Store,Vietnamese Restaurant,Warehouse Store,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Victoria Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [40]:
tr_york_grouped = tr_york_onehot.groupby('Neighborhood').mean().reset_index()
tr_york_grouped

,Neighborhood,Accessories Store,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Store,Bike Shop,Boutique,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Business Service,Butcher,Café,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Curling Ice,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Electronics Store,Falafel Restaurant,Fast Food Restaurant,Field,Fish & Chips Shop,Flea Market,Food & Drink Shop,Food Court,Food Service,Food Truck,Fried Chicken Joint,Furniture / Home Store,Gas Station,Gastropub,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health Food Store,Hobby Shop,Hockey Arena,Home Service,Hotel,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Korean Restaurant,Liquor Store,Lounge,Luggage Store,Massage Studio,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Movie Theater,Moving Target,Park,Pet Store,Pharmacy,Pizza Place,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Sandwich Place,Shoe Store,Shopping Mall,Skating Rink,Smoothie Shop,Spa,Sporting Goods Shop,Sports Bar,Supermarket,Sushi Restaurant,Thai Restaurant,Theater,Toy / Game Store,Trail,Video Game Store,Vietnamese Restaurant,Warehouse Store,Women's Store,Yoga Studio
0,"Bathurst Manor, Wilson Heights, Downsview North",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.095238,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.047619,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.095238,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.047619,0.000000,0.000000,0.000000,0.047619,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.0,0.00,0.047619,0.000000,0.047619,0.000000,0.000000,0.000000,0.047619,0.000000,0.000000,0.047619,0.000000,0.00,0.00,0.000000,0.047619,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.047619,0.000000,0.047619,0.000000,0.00,0.000000,0.000000,0.047619,0.047619,0.000000,0.000000,0.00,0.000000,0.000000,0.047619,0.000000,0.047619,0.000000,0.047619,0.0,0.000000,0.000000,0.000000,0.000000,0.047619,0.047619,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
1,Bayview Village,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.250000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Bedford Park, Lawrence Manor East",0.000000,0.000000,0.038462,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.038462,0.038462,0.0

### Above, you can find the full list of neighborhoods and their corresponding venue categories

In [41]:
tr_york_grouped.shape

(33, 119)

The formula below is used to determine the most common venues by neighborhood

In [42]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

## Below, the most common venue categories by neighborhood are listed

In [43]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = tr_york_grouped['Neighborhood']

for ind in np.arange(tr_york_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tr_york_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Pharmacy,Supermarket,Middle Eastern Restaurant,Diner,Health Food Store,Pizza Place,Restaurant,Sandwich Place
1,Bayview Village,Chinese Restaurant,Café,Bank,Japanese Restaurant,Accessories Store,Lounge,Moving Target,Movie Theater,Mobile Phone Shop,Miscellaneous Shop
2,"Bedford Park, Lawrence Manor East",Restaurant,Italian Restaurant,Coffee Shop,Sandwich Place,Pizza Place,Butcher,Café,Pub,Comfort Food Restaurant,Greek Restaurant
3,Caledonia-Fairbanks,Park,Women's Store,Pool,Juice Bar,Movie Theater,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Mexican Restaurant,Metro Station
4,"Del Ray, Mount Dennis, Keelsdale and Silverthorn",Convenience Store,Coffee Shop,Discount Store,Sandwich Place,Lounge,Moving Target,Movie Theater,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant


In [44]:
!pip uninstall -y numpy
!pip uninstall -y setuptools


Found existing installation: numpy 1.20.2
Uninstalling numpy-1.20.2:
  Successfully uninstalled numpy-1.20.2
Found existing installation: setuptools 56.0.0
Uninstalling setuptools-56.0.0:
  Successfully uninstalled setuptools-56.0.0


In [45]:
!pip install setuptools
!pip install numpy

  Using cached setuptools-56.0.0-py3-none-any.whl (784 kB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorboard 1.14.0 requires numpy>=1.12.0, which is not installed.
ibmdbpy 0.1.6 requires numpy, which is not installed.
altair 4.1.0 requires numpy, which is not installed.
  Using cached numpy-1.20.2-cp37-cp37m-manylinux2010_x86_64.whl (15.3 MB)


In [46]:
# import k-means from clustering stage
!pip install -U scikit-learn

# Clustering

In [47]:
from sklearn.cluster import KMeans

## 4 Clusters are used to correlate results to the 4 boroughs and to notice differences

In [48]:
# set number of clusters
kclusters = 4

tr_york_grouped_clustering = tr_york_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tr_york_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 0, 1, 1, 1, 1, 1, 1], dtype=int32)

In [49]:
kmeans

KMeans(n_clusters=4, random_state=0)

In [50]:
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Pharmacy,Supermarket,Middle Eastern Restaurant,Diner,Health Food Store,Pizza Place,Restaurant,Sandwich Place
1,Bayview Village,Chinese Restaurant,Café,Bank,Japanese Restaurant,Accessories Store,Lounge,Moving Target,Movie Theater,Mobile Phone Shop,Miscellaneous Shop
2,"Bedford Park, Lawrence Manor East",Restaurant,Italian Restaurant,Coffee Shop,Sandwich Place,Pizza Place,Butcher,Café,Pub,Comfort Food Restaurant,Greek Restaurant
3,Caledonia-Fairbanks,Park,Women's Store,Pool,Juice Bar,Movie Theater,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Mexican Restaurant,Metro Station
4,"Del Ray, Mount Dennis, Keelsdale and Silverthorn",Convenience Store,Coffee Shop,Discount Store,Sandwich Place,Lounge,Moving Target,Movie Theater,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant


## Cluster labels are added below

In [51]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_,allow_duplicates=True)

neighborhoods_venues_sorted.head()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Pharmacy,Supermarket,Middle Eastern Restaurant,Diner,Health Food Store,Pizza Place,Restaurant,Sandwich Place
1,1,Bayview Village,Chinese Restaurant,Café,Bank,Japanese Restaurant,Accessories Store,Lounge,Moving Target,Movie Theater,Mobile Phone Shop,Miscellaneous Shop
2,1,"Bedford Park, Lawrence Manor East",Restaurant,Italian Restaurant,Coffee Shop,Sandwich Place,Pizza Place,Butcher,Café,Pub,Comfort Food Restaurant,Greek Restaurant
3,0,Caledonia-Fairbanks,Park,Women's Store,Pool,Juice Bar,Movie Theater,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Mexican Restaurant,Metro Station
4,1,"Del Ray, Mount Dennis, Keelsdale and Silverthorn",Convenience Store,Coffee Shop,Discount Store,Sandwich Place,Lounge,Moving Target,Movie Theater,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant


In [52]:
tr_york_merged = tr_york_br

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
tr_york_merged = tr_york_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

tr_york_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,br_n,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,2,0.0,Food & Drink Shop,Park,Accessories Store,Korean Restaurant,Movie Theater,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Mexican Restaurant,Metro Station
1,M4A,North York,Victoria Village,43.725882,-79.315572,2,1.0,Coffee Shop,Pizza Place,Hockey Arena,Portuguese Restaurant,Liquor Store,Movie Theater,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Mexican Restaurant
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,2,1.0,Clothing Store,Accessories Store,Furniture / Home Store,Coffee Shop,Boutique,Miscellaneous Shop,Vietnamese Restaurant,Women's Store,Mobile Phone Shop,Movie Theater
3,M3B,North York,Don Mills North,43.745906,-79.352188,2,1.0,Caribbean Restaurant,Gym,Athletics & Sports,Japanese Restaurant,Café,Middle Eastern Restaurant,Massage Studio,Mediterranean Restaurant,Metro Station,Mexican Restaurant
4,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,3,1.0,Pizza Place,Bank,Pharmacy,Gym / Fitness Center,Flea Market,Intersection,Breakfast Spot,Pet Store,Athletics & Sports,Gastropub


In [53]:
tr_york_merged['Cluster Labels'].head()

0    0.0
1    1.0
2    1.0
3    1.0
4    1.0
Name: Cluster Labels, dtype: float64

### Cluster labels are converted to integers below to allow for formulas further down to run

In [54]:
tr_york_merged.fillna({'Cluster Labels':0.0}, inplace=True)
#tr_york_merged.astype({'Cluster Labels': int})
tr_york_merged = tr_york_merged.astype({'Cluster Labels': int})
tr_york_merged

,PostalCode,Borough,Neighborhood,Latitude,Longitude,br_n,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,2,0,Food & Drink Shop,Park,Accessories Store,Korean Restaurant,Movie Theater,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Mexican Restaurant,Metro Station
1,M4A,North York,Victoria Village,43.725882,-79.315572,2,1,Coffee Shop,Pizza Place,Hockey Arena,Portuguese Restaurant,Liquor Store,Movie Theater,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Mexican Restaurant
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,2,1,Clothing Store,Accessories Store,Furniture / Home Store,Coffee Shop,Boutique,Miscellaneous Shop,Vietnamese Restaurant,Women's Store,Mobile Phone Shop,Movie Theater
3,M3B,North York,Don Mills North,43.745906,-79.352188,2,1,Caribbean Restaurant,Gym,Athletics & Sports,Japanese Restaurant,Café,Middle Eastern Restaurant,Massage Studio,Mediterranean Restaurant,Metro Station,Mexican Restaurant
4,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,3,1,Pizza Place,Bank,Pharmacy,Gym / Fitness Center,Flea Market,Intersection,Breakfast Spot,Pet Store,Athletics & Sports,Gastropub
5,M6B,North York,Glencairn,43.709577,-79.445073,2,1,Bakery,Pizza Place,Park,Japanese Restaurant,Asian Restaurant,Metro Station,Luggage Store,Massage Studio,Mediterranean Restaurant,Accessories Store
6,M3C,North York,Don Mills South,43.725900,-79.340923,2,1,Coffee Shop,Restaurant,Gym,Italian Restaurant,Supermarket,Bike Shop,Beer Store,Clothing Store,Sandwich Place,Dim Sum Restaurant
7,M4C,East York,Woodbine Heights,43.695344,-79.318389,3,1,Intersection,Park,Curling Ice,Beer Store,Skating Rink,Accessories Store,Lounge,Luggage Store,Massage Studio,Metro Station
8,M6C,York,Humewood-Cedarvale,43.693781,-79.428191,4,1,Moving Target,Hockey Arena,Trail,Field,Korean Restaurant,Movie Theater,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Mexican Restaurant
9,M6E,York,Caledonia-Fairbanks,43.689026,-79.453512,4,0,Park,Women's Store,Pool,Juice Bar,Movie Theater,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Mexican Restaurant,Metro Station


### Clusters are mapped below

In [55]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tr_york_merged['Latitude'], tr_york_merged['Longitude'], tr_york_merged['Neighborhood'], tr_york_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### It can be seen that the clusters do not correspond to the boroughs as shown earlier. Two clusters only have one entry. The entry for East York / East Toronto (it's own borough) is now part of cluster 0. Cluster 1 has by far the most entries. Cluster 2 (cyan) and 3 (light green, top left, difficult to make out) only have one entry each. In the following, the clusters are explored.

In [56]:
tr_york_merged.loc[tr_york_merged['Cluster Labels'] == 0, tr_york_merged.columns[[1] + list(range(5, tr_york_merged.shape[1]))]]

,Borough,br_n,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,2,0,Food & Drink Shop,Park,Accessories Store,Korean Restaurant,Movie Theater,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Mexican Restaurant,Metro Station
9,York,4,0,Park,Women's Store,Pool,Juice Bar,Movie Theater,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Mexican Restaurant,Metro Station
16,East York/East Toronto,1,0,Park,Convenience Store,Intersection,Accessories Store,Liquor Store,Movie Theater,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Mexican Restaurant
19,North York,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,North York,2,0,Park,Accessories Store,Juice Bar,Movie Theater,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Mexican Restaurant,Metro Station,Mediterranean Restaurant
31,York,4,0,Park,Accessories Store,Juice Bar,Movie Theater,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Mexican Restaurant,Metro Station,Mediterranean Restaurant
32,North York,2,0,Convenience Store,Bar,Park,Korean Restaurant,Moving Target,Movie Theater,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Mexican Restaurant


Cluster 0 is primarily made up of parks. This differentiates it from the other clusters and explains why the venues are spread across the city.

In [57]:
tr_york_merged.loc[tr_york_merged['Cluster Labels'] == 1, tr_york_merged.columns[[1] + list(range(5, tr_york_merged.shape[1]))]]

,Borough,br_n,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,2,1,Coffee Shop,Pizza Place,Hockey Arena,Portuguese Restaurant,Liquor Store,Movie Theater,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Mexican Restaurant
2,North York,2,1,Clothing Store,Accessories Store,Furniture / Home Store,Coffee Shop,Boutique,Miscellaneous Shop,Vietnamese Restaurant,Women's Store,Mobile Phone Shop,Movie Theater
3,North York,2,1,Caribbean Restaurant,Gym,Athletics & Sports,Japanese Restaurant,Café,Middle Eastern Restaurant,Massage Studio,Mediterranean Restaurant,Metro Station,Mexican Restaurant
4,East York,3,1,Pizza Place,Bank,Pharmacy,Gym / Fitness Center,Flea Market,Intersection,Breakfast Spot,Pet Store,Athletics & Sports,Gastropub
5,North York,2,1,Bakery,Pizza Place,Park,Japanese Restaurant,Asian Restaurant,Metro Station,Luggage Store,Massage Studio,Mediterranean Restaurant,Accessories Store
6,North York,2,1,Coffee Shop,Restaurant,Gym,Italian Restaurant,Supermarket,Bike Shop,Beer Store,Clothing Store,Sandwich Place,Dim Sum Restaurant
7,East York,3,1,Intersection,Park,Curling Ice,Beer Store,Skating Rink,Accessories Store,Lounge,Luggage Store,Massage Studio,Metro Station
8,York,4,1,Moving Target,Hockey Arena,Trail,Field,Korean Restaurant,Movie Theater,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Mexican Restaurant
10,East York,3,1,Coffee Shop,Furniture / Home Store,Burger Joint,Bank,Sporting Goods Shop,Bike Shop,Restaurant,Sandwich Place,Liquor Store,Shopping Mall
11,North York,2,1,Fast Food Restaurant,Mediterranean Restaurant,Golf Course,Dog Run,Athletics & Sports,Pool,Liquor Store,Lounge,Luggage Store,Massage Studio


Cluster 1 shows a wide variety of restaurants and shops.

In [58]:
tr_york_merged.loc[tr_york_merged['Cluster Labels'] == 2, tr_york_merged.columns[[1] + list(range(5, tr_york_merged.shape[1]))]]

,Borough,br_n,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
27,North York,2,2,Food Service,Baseball Field,Accessories Store,Juice Bar,Movie Theater,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Mexican Restaurant,Metro Station


In [59]:
tr_york_merged.loc[tr_york_merged['Cluster Labels'] == 3, tr_york_merged.columns[[1] + list(range(5, tr_york_merged.shape[1]))]]

,Borough,br_n,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,North York,2,3,Pizza Place,Accessories Store,Park,Movie Theater,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Mexican Restaurant,Metro Station,Mediterranean Restaurant


Cluster 2 and 3 are primarily distinguished through having the only food service and being far detached from the other venues.

## How could this have been improved?

As could be seen, cluster 1 takes too many venues compared to the others, especially 2 and 3. By increasing the range of the radius and the amount of venues, and by selecting different boroughs, the clustering could have become more meaningful.